In [7]:
import serial
import crcmod
import struct
import sys
import time
import threading

# Replace this with the absolute path to the directory containing your module
module_dir = '/Users/kpt/Code/pyRelay'


# Add the directory to sys.path
if module_dir not in sys.path:
    sys.path.append(module_dir)

# Now you can import your module (replace 'module' with the name of your Python file without the .py extension)
from zsrelay import ZsRelay


In [2]:
# list out the serial ports
def list_ports():
    import serial.tools.list_ports
    ports = serial.tools.list_ports.comports()
    for port in ports:
        print(port)

list_ports()

/dev/cu.wlan-debug - n/a
/dev/cu.Bluetooth-Incoming-Port - n/a
/dev/cu.usbserial-A552B77CF6 - M5stack


In [3]:
baudrate = 38400
port = '/dev/cu.usbserial-A552B77CF6' #insert the port in string format
dev_pos = 1

zsr = ZsRelay(baudrate, port, dev_pos)
zsr.openSerial()

Serial<id=0x1073822f0, open=True>(port='/dev/cu.usbserial-A552B77CF6', baudrate=38400, bytesize=8, parity='N', stopbits=1, timeout=1, xonxoff=False, rtscts=False, dsrdtr=False)

In [4]:
# Initialise relay controller with an all off switch command
all_off_s = '01 06 00 34 00 00 C8 04' #needs formatting to bytes while removing spaces
all_off_b = bytes.fromhex(all_off_s) #converts hex string to bytes also handles spaces ' '
zsr.writeByte(all_off_b)

b''

In [12]:
# for the range of 0 to 7 (8 relays) turn them on one by one
# channel 0 is on for x seconds then off
# channel 1 is on for x seconds then off
experiment = False
experiment_lock = threading.Lock()

def loop_exp(start_channel: int, end_channel: int, delay_s: int):
    global experiment
    with experiment_lock:
        experiment = True
    for i in range(start_channel, end_channel):
        print('Turning on relay ' + str(i))
        zsr.turnOnRelay(i)
        time.sleep(delay_s)
        print('Turning off relay ' + str(i))
        zsr.turnOffRelay(i)
        time.sleep(delay_s)
    with experiment_lock:
        experiment = False
    
    print('Experiment complete')
    

exp_thread = threading.Thread(target=loop_exp, args=(1, 5, 5))  # Replace these arguments with your desired values

# Start the thread
exp_thread.start()

Turning on relay 1


Turning off relay 1


In [41]:
# Example of another function that checks the experiment state
def check_experiment_state():
    with experiment_lock:
    # Here you can add logic to perform actions based on the experiment state
        if experiment:
            print("Experiment is running")
        else:
            print("Experiment is not running")

In [42]:
check_experiment_state()

Experiment is not running
